# HydroTrend

* Link to this notebook: https://github.com/csdms/pymt/blob/master/notebooks/hydrotrend.ipynb
* Package installation command: `$ conda install notebook pymt_hydrotrend`
* Command to download a local copy:

  `$ curl -O https://raw.githubusercontent.com/csdms/pymt/master/notebooks/hydrotrend.ipynb`
  
HydroTrend is a 2D hydrological water balance and transport model that simulates water discharge and sediment load at a river outlet. You can read more about the model, find references or download the C source code at: https://csdms.colorado.edu/wiki/Model:HydroTrend.

This notebook has been created by Irina Overeem, September 18, 2019.


### River Sediment Supply Modeling

This notebook is meant to give you a better understanding of what HydroTrend is capable of. In this example we are using a theoretical river basin of ~1990 km<sup>2</sup>, with 1200m of relief and a river length of
~100 km. All parameters that are shown by default once the HydroTrend Model is loaded are based
on a present-day, temperate climate. Whereas these runs are not meant to be specific, we are
using parameters that are realistic for the [Waiapaoa River][map_of_waiapaoa] in New Zealand. The Waiapaoa River
is located on North Island and receives high rain and has erodible soils, so the river sediment
loads are exceptionally high. It has been called the *"dirtiest small river in the world"*.


A more detailed description of applying HydroTrend to the Waipaoa basin, New Zealand has been published in WRR: [hydrotrend_waipaoa_paper]. 


[map_of_waiapaoa]: https://www.google.com/maps/place/Waipaoa+River/@-38.5099042,177.7668002,71814m/data=!3m1!1e3!4m5!3m4!1s0x6d65def908624859:0x2a00ef6165e1dfa0!8m2!3d-38.5392405!4d177.8843782
[hydrotrend_presentation]: https://csdms.colorado.edu/wiki/File:SedimentSupplyModeling02_2013.ppt
[hydrotrend_waipaoa_paper]: http://dx.doi.org/10.1029/2006WR005570

### Run HydroTrend Simulations with pymt



Now we will be using the capability of the Python Modeling Tool, pymt. Pymt is a Python toolkit for running and coupling Earth surface models. 

https://csdms.colorado.edu/wiki/PyMT


In [1]:
# To start, import numpy and matplotlib.
import matplotlib.pyplot as plt
import numpy as np


In [2]:
# Then we import the package 
import pymt.models

hydrotrend = pymt.models.Hydrotrend()

➡ models: Avulsion, Plume, Sedflux3D, Subside, Rafem, PRMSSurface, PRMSStreamflow, PRMSSoil, PRMSGroundwater, FrostNumber, Ku, Hydrotrend, GIPL, ECSimpleSnow, Cem, Waves


In [3]:
import pymt
pymt.__version__

'v1.1.3'

## Learn about the Model Input
<br>
HydroTrend will now be activated in PyMT. You can find information on the model, the developer, the papers that describe the moel in more detail etc. 
Importantly you can scroll down a bit to the Parameters list, it shows what parameters the model uses to control the simulations. The list is alphabetical and uses precisely specified 'Standard Names'.
Note that every parameter has a 'default' value, so that when you do not list it in the configure command, you will run with these values.


In [4]:
# Get basic information about the HydroTrend model 
help(hydrotrend)

Help on Hydrotrend in module pymt.framework.bmi_bridge object:

class Hydrotrend(BmiCap)
 |  Basic Model Interface for HydroTrend.
 |  
 |  Climate-driven hydrological water balance and transport model that
 |  simulates water discharge and sediment load at a river outlet.
 |  HydroTrend simulates water and sediment fluxes at a daily timescale
 |  based on drainage basin characteristics and climate. HydroTrend can
 |  provide this river flux information to other components like CEM and
 |  Sedflux2D or Sedflux3D
 |  
 |  Author:
 |  - Albert Kettner
 |  Version: 3.0.1
 |  License: GPLv3
 |  DOI: 10.1594/IEDA/100135
 |  URL: http://csdms.colorado.edu/wiki/Model_help:HydroTrend
 |  
 |  Cite as:
 |  
 |      @article{kettner2008hydrotrend,
 |        title={HydroTrend v. 3.0: a climate-driven hydrological transport model that simulates discharge and sediment load leaving a river system},
 |        author={Kettner, Albert J and Syvitski, James PM},
 |        journal={Computers \& Geoscienc

### Exercise 1: Explore the Hydrotrend base-case river simulation

For this case study, first we will create a subdirectory in which the basecase (BC) simulation will be implemented. 

Then we specify for how long we will run a simulation: for 100 years at daily time-step.
This means you run Hydrotrend for 36,500 days total. 
This is also the line of code where you would add other input parameters with their values.

In [5]:
# Set up Hydrotrend model by indicating the number of years to run
config_file, config_folder = hydrotrend.setup("_hydrotrendBC", run_duration=100)

With the cat command you can print character by character one of the two input files that HydroTrend uses.
HYDRO0.HYPS: This first file specifies the River Basin Hysometry - the surface area per elevation zone. The hypsometry captures the geometric characteristics of the river basin, how high is the relief, how much uplands are there versus lowlands, where would the snow fall elevation line be et cetera. <br>
HYDRO.IN: This other file specifies the basin and climate input data. 

In [6]:
cat  _hydrotrendBC/HYDRO0.HYPS

-------------------------------------------------
Hypsometry input file for HYDROTREND
First line: number of hypsometric bins
Other lines: altitude (m) and area in (km^2) data
-------------------------------------------------
25
1	208.9168081
51	375.2451287
101	536.0996727
151	727.1589437
201	902.604255
251	1096.567651
301	1257.078121
351	1414.457889
401	1529.215605
451	1630.798354
501	1704.457117
551	1769.103192
601	1816.557667
651	1862.242091
701	1898.728609
751	1932.088621
801	1954.393164
851	1969.016904
901	1977.0094
951	1981.623038
1001	1984.268222
1051	1985.923785
1101	1986.740178
1151	1986.92
1201	1986.921

In [7]:
cat _hydrotrendBC/HYDRO.IN

Waipaoa 50yrs present
ON				2) Write output to ASCII files (ON/OFF)
./HYDRO_OUTPUT/		3) Location where the output data will be stored (not optional for web)
1				4) No. of epochs to run (leave 1 line blank between epochs; start copying from nr 5)
0 100 d		5) Start year; no. of years to run; averaging interval: D,M,S or Y
4				6) Number of suspended sed. grain sizes to simulate (max 10)
.2 .2 .25 .35	7) Proportion of sediment in each grain size (sum=1)
14.26 0.0 0.55  8) Yrly Tbar: start (C), change/yr (C/a), std dev
1.59 0.0 0.3 9) Yrly P sum: start (m/a), change/yr (m/a/a), std. dev (m).
1. 1.9 7	10) Rain: Mass Balance Coef, Distribution Exp, Distribution Range
1.1		11) Constant annual base flow (m^3/s)
Jan  19.14  1.05 103.91 62.07 12-23) monthly climate variables
Feb  18.85  1.32 101.22 69.90 column  variable	description
Mar  17.49  1.07 121.90 71.12 ------  --------	-----------
Apr  14.76  0.88 147.96 56.59 1	moname  	month name (not used)
May  12.08  0.86 125.50 71.91 2	tmpinm  	m

In [8]:
#In pymt one can always find out what output a model generates by using the .output_var_names method. 
hydrotrend.output_var_names

('atmosphere_bottom_air__domain_mean_of_temperature',
 'channel_exit_water_sediment~suspended__mass_flow_rate',
 'channel_exit_water_flow__speed',
 'channel_entrance_water_sediment~bedload__mass_flow_rate',
 'channel_exit_water__volume_flow_rate',
 'channel_exit_water_x-section__width',
 'channel_exit_water_x-section__depth',
 'channel_entrance_water__volume_flow_rate',
 'atmosphere_water__domain_mean_of_precipitation_leq-volume_flux',
 'channel_exit_water_sediment~bedload__mass_flow_rate',
 'channel_exit_water_sediment~suspended__mass_concentration')

In [9]:
# Now we initialize the model with the configure file and in the configure folder
hydrotrend.initialize(config_file, config_folder)

In [10]:
# this line of code lists time parameters, when, how long and at what timestep will the model simulation work?
hydrotrend.start_time, hydrotrend.time, hydrotrend.end_time, hydrotrend.time_step, hydrotrend.time_units

#so what is our model time-step? How long does it run for?

(0.0, 0.0, 36500.0, 1.0, 'd')

In [11]:
# this code declares numpy arrays for several important parameters we want to save.
n_days = int(hydrotrend.end_time)
q = np.empty(n_days) #river discharge at the outlet
qs = np.empty(n_days)# sediment load at the outlet
cs = np.empty(n_days) # suspended sediment concentration for different grainsize classes at the outlet
qb = np.empty(n_days) # bedload at the outlet

# here we have coded up the time loop using i as the index
# we update the model with one timestep at the time, untill we reach the end time 
# for each time step we also get the values for the output parameters we wish to 
for i in range(n_days):
    hydrotrend.update()
    q[i] = hydrotrend.get_value("channel_exit_water__volume_flow_rate")
    qs[i] = hydrotrend.get_value("channel_exit_water_sediment~suspended__mass_flow_rate")
    cs[i] = hydrotrend.get_value("channel_exit_water_sediment~suspended__mass_concentration")
    qb[i] = hydrotrend.get_value("channel_exit_water_sediment~bedload__mass_flow_rate")

In [ ]:
# We can plot the simulated output timeseries of Hydrotrend, for example the river discharge

plt.plot(q)
plt.title('HydroTrend simulation of 100 year river discharge, Waiapaoa River')
plt.ylabel('river discharge in m3/sec')
plt.show

In [ ]:
# Or you can plot a subset of the simulated daily timeseries using the index

#for example the first year
plt.plot(q[0:365], 'black')
# compare with the last year
plt.plot(q[-366:-1],'grey')
plt.title('HydroTrend simulation of first and last year discharge, Waiapaoa River')
plt.show()


In [ ]:
# Of course, it is important to calculate statistical properties of the simulated parameters

print(q.mean())
hydrotrend.get_var_units("channel_exit_water__volume_flow_rate")

 ## <font color = green> Assignment 1 </font> 

Calculate mean water discharge Q, mean suspended load Qs, mean sediment concentration Cs, and mean bedload Qb for this 100 year simulation of the river dynamics of the Waiapaoa River.
Note all values are reported as daily averages. What are the units?


In [ ]:
# your code goes here
#hint look several cells up where we create the variables q, qs, cs, and qb.


## <font color = green> Assignment 2 </font> 

Identify the highest flood event for this simulation. Is this the 100-year flood? Please list a definition of a 100 year flood, and discuss whether the modeled extreme event fits this definition. 
Plot the year of Q-data which includes the flood.


In [ ]:
# here you can calculate the maximum river discharge.


#how can we also anaylze peak floods and extreme flood prediction? 
#What if the model only ran for 30 years? How would you calculate the 100-year flood?

In [ ]:
#let's look at one way to find the DAY associated with the max flood found above.

#let's make a new array
a = np.array([5, 12, 1, 0, 6, -1, 15, 29, -5 ])
print(a)
#where is the maximum value located? What index?

imax = np.argmax(a)
print(imax)
print(a[imax])

In [ ]:
# your code to determine which day and which year encompass the maximum discharge go here
# Hint: you will want to determine the index of this day first, look into the numpy.argmax and numpy.argmin 

#so what day is the max flow found? How would this change if the time-step was every 2 days?


# as a sanity check, plot the max. discharge ontop of the entire timeseries of flow


## <font color = green> Assignment 3 </font> 

Calculate the mean annual sediment load for this river system.


In [ ]:
# your code goes here
# you will have to sum all days of the individual years, to get the annual loads, 
# then calculate the mean over the 100 years.

# one possible trick is to use the .reshape() method to change from one long vector 36,500 items long to one that is
# 100 rows (1 row per year) with 365 columns (a column per day)

#now calc. the SUM of this matrix: 
#remember you want one value per row! so summing over all columns


# plot a graph of the 100 years timeseries of the total annual loads 



In [ ]:
# take the mean over the 100 years

#what are the units of this value??

### HydroTrend Exercise 2: How does a river system respond to climate change; two simple scenarios for the coming century.

Now we will look at changing climatic conditions in a small river basin. We'll change temperature and precipitation regimes and compare discharge and sediment load characteristics to the original basecase. And we will look at the are potential implications of changes in the peak events.

Modify the mean annual temperature T, the mean annual precipitation P. You can specify trends over time, by modifying the parameter ‘change in mean annual temperature’ or ‘change in mean annual precipitation’. HydroTrend runs at daily timestep, and thus can deal with seasonal variations in temperature and precipitation for a basin. The model ingests monthly mean input values for these two climate parameters and their monthly standard deviations, ideally the values would be derived from analysis of a longterm record of daily climate data. You can adapt seasonal trends by using the monthly values.

## <font color = green> Assignment 4 </font> 
What happens to river discharge, suspended load and bedload if the mean annual temperature in this specific river basin increases by 4 °C over the next 50 years? In this assignment we set up a new simulation for a warming climate.

In [ ]:
# Set up a new run of the Hydrotrend model 
# Create a new config file a different folder for input and output files, 
# indicating the number of years to run, and specify the change in mean annual temparture parameter
hydrotrendHT = pymt.models.Hydrotrend()
config_file, config_folder = hydrotrendHT.setup("_hydrotrendhighT", run_duration=50,  change_in_mean_annual_temperature=0.08)

In [ ]:
# intialize the new simulation
hydrotrendHT.initialize(config_file, config_folder)

In [ ]:
# the code for the timeloop goes here
# I use the abbrevation HT for 'High Temperature' scenario
n_days_HT = int(hydrotrendHT.end_time)
q_HT = np.empty(n_days_HT) #river discharge at the outlet
qs_HT = np.empty(n_days_HT)# sediment load at the outlet
cs_HT = np.empty(n_days_HT) # suspended sediment concentration for different grainsize classes at the outlet
qb_HT = np.empty(n_days_HT) # bedload at the outlet
for i in range(n_days_HT):
    hydrotrendHT.update() #what does this line of code do??
    q_HT[i] = hydrotrendHT.get_value("channel_exit_water__volume_flow_rate")
    qs_HT[i] = hydrotrendHT.get_value("channel_exit_water_sediment~suspended__mass_flow_rate")
    cs_HT[i] = hydrotrendHT.get_value("channel_exit_water_sediment~suspended__mass_concentration")
    qb_HT[i] = hydrotrendHT.get_value("channel_exit_water_sediment~bedload__mass_flow_rate")

In [ ]:
#let's plot our results of river discharge for this new model


In [ ]:
#now plot the last year of discharge for the new model over the first year of discharge for the new model


#what do you see?

In [ ]:
# your code that prints out the mean annual river discharge, 
# remember how many years we run this simulation for! and remember np.reshape()


#the mean annual sediment load and the mean bedload goes here


# print out these same parameters for the basecase for comparison



In [ ]:
#now plot these annual values on top of each other from NEW model vs. old model for all 3 values


## <font color = green> Assignment 5 </font> 

So what is the effect of a warming basin temperature? 
How much increase or decrease of river discharge do you see after 50 years? <br>
How is the mean suspended load affected? <br>
How does the mean bedload change? <br>
What happens to the peak event; look at the maximum sediment load event of the last 5 years of the simulation?

In [ ]:
# type your answers here

## <font color = green> Assignment 6 </font>

In addition, climate model predictions indicate that perhaps precipitation intensity and variability could increase. How would you possibly model this? Discuss how you would modify your input settings for precipitation.



In [ ]:
#type your answer here

## <font color = green> Assignment 7 </font>
Now go back and make sure ALL the graphs have labels, units, titles, etc. Also check that you have put comments in and answered all questions asked!

### Exercise 3: How do humans affect river sediment loads?

Here we will look at the effect of human in a river basin. Humans can accelerate erosion
processes, or reduce the sediment loads traveling through a river system. Both concepts can
be simulated, first run 3 simulations systematically increasing the anthropogenic factor (0.5-8.0 is the range).

In [ ]:
#add your code to create THREE new simulations (don't overwrite any with the same name)

## <font color = green> Assignment 8 </font>
Describe in your own words the meaning of the human-induced erosion factor, (Eh). This factor is parametrized as the  “Anthropogenic” factor in HydroTrend. Read more about this in: Syvitski & Milliman, 2007, Geology, Geography, and Humans Battle for Dominance over the Delivery of Fluvial Sediment to the Coastal Ocean. 2007, 115, p. 1–19. 

In [ ]:
# your explanation goes here, can you list two reasons why this factor would be unsuitable or it would fall short?

## <font color = green> Bonus Assignment 9 </font>
Model a scenario of a drinking water supply reservoir to be planned in the coastal area of the basin. The reservoir would have 800 km2 of contributing drainage area and be 3 km long, 200m wide and 100m deep. Set up a simulation with these parameters.

In [ ]:
# Set up a new 50 year of the Hydrotrend model 
# Create a new directory, and a config file indicating the number of years to run, and specify different reservoir parameters

In [ ]:
# initialize the new simulation

In [ ]:
# your code for the timeloop and update loop goes here

In [ ]:
# plot a bar graph comparing Q mean, Qs mean, Qmax, Qs Max, Qb mean and Qbmax for the basecase run and the reservoir run

In [ ]:
# Describe how such a reservoir affects the water and sediment load at the coast (i.e. downstream of the reservoir)?

## <font color = green> Bonus Assignment 10 </font>
Set up a simulation for a different river basin. 
This means you would need to change the HYDRO0.HYPS file and change some climatic parameters. 
There are several hypsometric files packaged with HydroTrend, you can use one of those, but are welcome to do something different!

In [ ]:
# write a short motivation and description of your scenario

In [ ]:
# make a 2 panel plot using the subplot functionality of matplotlib 
# One panel would show the hypsometry of the Waiapohoa and the other panel the hypsometry of your selected river basin 

In [ ]:
# Set up a new 50 year of the Hydrotrend model 
# Create a new directory for this different basin

In [ ]:
# initialize the new simulation

In [ ]:
# your code for the timeloop and update loop goes here

In [ ]:
# plot a line graph comparing Q mean, Qs mean, for the basecase run and the new river basin run

## <font color = green>  ALL DONE! </font>